In [1]:
import pandas as pd
from sklearn import linear_model
import numpy as np
np.set_printoptions(suppress=True)
from sklearn import preprocessing
import operator
import json

In [2]:
data = pd.read_excel('../scratch/obce_vystup_v02_omezene.xlsx')

In [3]:
koef = pd.read_excel('../scratch/koeficientyPorovnani_v03omez.xlsx', skip_footer=2).set_index('Faktor')

In [4]:
koef['Průměr'] = round(koef['Průměr'] / 100, 5)

In [5]:
nms = {
    'Nezaměstnanost': 'nezam',
    'Emise': 'emise',
    'Naděje dožití mužů': 'doziti_m',
    'Index rozvodovosti': 'rozvody',
    'Nedostuopnost MŠ': 'ned_ms',
    'Nedostupnost SŠ': 'ned_ss',
    'Nedostupnost zdrav. zař.': 'ned_zdrav',
    'Exekucí na obyv.': 'exe',
    'Podíl prac. v průmyslu': 'prumysl',
    'Bezpečnost': 'bezpecnost',
    'Přírůstek obyvatel': 'prirust',
    'Procento věřících': 'verici',
    'Nedostupnost internetu': 'ned_net',
    'Vzdálenost k okresnímu městu': 'vz_okr',
}

In [6]:
koef = koef[['Průměr']].to_dict()['Průměr']

In [ ]:
kout = {}
for  k in koef:
    if k in nms:
        kout[nms[k]] = koef[k]

In [ ]:
kout

In [7]:
# standardizace a zaokrouhleni
for n in nms.keys():
    data[n] = pd.Series(preprocessing.scale(data[n].values))
    data[n] = data[n].apply(lambda x: round(x, 5))

c:\program files\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [8]:
# bude se aplikovat az na klientovi
for k in koef:
    if k in data.columns:
        data[k] = data[k].apply(lambda x: x * koef[k])

In [9]:
nms.update({
    'Název obce': 'nazev',
    'Název okresu': 'okres'})
data.rename(index=str, columns=nms, inplace=True)

In [10]:
data.set_index('NUTS5 (Obec)', inplace=True)

In [ ]:
with open('../scratch/data.js', 'w', encoding='utf-8') as f:
    f.write('var data = ' + json.dumps(data[list(nms.values())].to_dict(orient='index')) + ';')

In [11]:
sel = list(nms.values())
sel.remove('nazev')
sel.remove('okres')

In [12]:
data['kvalita'] = 1 - (data[sel].sum(axis=1) + 0.50519)

In [13]:
# naskalovani vysledku na 0 - 100
mini = data.kvalita.min()
maxi = data.kvalita.max()

def norm(v):
    return ((v - mini) / (maxi - mini)) * 100

data.kvalita = data.kvalita.apply(lambda x: norm(x))